In [33]:
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from datetime import datetime, timedelta
import argparse
import psycopg2
import psycopg2.extras as extras
import pickle

from math import *
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from db.postgres import *

In [30]:
PG_USER = "postgres"
PG_PASSWORD = "1"
PG_DATABASE = "stock"
PG_HOST = "172.18.0.3"
PG_PORT = 5432
PG_DRIVER = 'org.postgresql.Driver'

start_date = '2015-01-01'
end_date = '2023-02-01'

In [32]:
class DataHandler:

    '''
    '''

    def __init__(self, ticket, lookback = 60):
        self.ticket = ticket
        self.lookback = lookback


    def get_scaler(self):
        df = psql.read_sql(f"select date, close from stock_raw where stock = '{self.ticket}'", connect())
        dataset = df[['close']].values

        scaler = MinMaxScaler(feature_range=(0,1))
        dataset = scaler.fit_transform(dataset)

        self.dataset = dataset

        return scaler


    def model_input(self):

        self.get_scaler()

        dataX, dataY = [], []
        for i in range(len(self.dataset) - self.lookback):
            row = [a for a in self.dataset[i:i+self.lookback]]
            dataX.append(row)
            dataY.append(self.dataset[i + self.lookback][0])

        X, y = np.array(dataX), np.array(dataY)

        # Train-test split
        split_point = int(len(self.dataset)*0.9)
        X_train, y_train = X[:split_point], y[:split_point]
        X_val, y_val = X[split_point:len(X)], y[split_point:len(y)]

        return X_train, y_train, X_val, y_val

In [31]:
def load_model(ticket):
    return pickle.load(open(f'model/LTSM_{ticket}.sav', 'rb'))

def update_rmse(ticket, run_date):

    '''
    '''
    model = DataHandler(ticket=ticket, lookback=60)

    X_train, y_train, X_val, y_val = model.preprocess()

    ltsm = load_model(ticket)
    y_predict = ltsm.predict(X_val)

    scaler = model.get_scaler()
    predict_price = scaler.inverse_transform(y_predict.reshape(-1, 1))
    true_price = scaler.inverse_transform(y_val.reshape(-1, 1))

    rmse = sqrt(mean_squared_error(true_price, predict_price))

    postgres_operator(
        query=f"""
            delete from rmse where date = '{run_date}';
            insert into rmse values ({run_date}, {ticket}, {rmse});
        """,
        conn=connect()
    )